### Using Local Files

In [1]:
from csv import reader, writer

In [2]:
def read_local_file(file_name):
    if '.csv' in file_name:
        rdr = reader(open(file_name, 'rb'))
        return rdr
    return open(file_name, 'rb')

def write_local_file(file_name, data):
    with open(file_name, 'wb') as open_file:
        if type(data) is list:
            wr = writer(open_file)
            for line in data:
                wr.writerow(line)
        else:
            open_file.write(data)

In [3]:
# an example where we need the most recent file in a directory.
import os

def get_latest(folder):
    files = [os.path.join(folder, f) for f in os.listdir(folder)]
    files.sort(key=lambda x:os.path.getmtime(x), reverse = True)
    return files[0]

In [4]:
# parsing config file
import configparser

def get_config(env):
    config = configparser.ConfigParser()
    if env == 'PROD':
        return config.read(['config/production.cfg'])
    elif env == 'TEST':
        return config.read(['config/test.cfg']) 
    return config.read(['config/development.cfg'])

In [5]:
# from some_api import get_client

def api_login():
    config = get_config('PROD')
    my_client = get_client(config.get('api_login', 'user'), config.get('api_login', 'auth_key'))
    return my_client

In [6]:
# parsing the list of info 
config = configparser.ConfigParser()
config.read(['config/email.cfg', 'config/database.cfh', 'config/staging.cfg'])

[]

### Command-line arguments

In [7]:
# python my_script.py DEV

In [8]:
# from import_config import get_config
import sys

In [9]:
def main(env):
    config = get_config(env)
    print(config)
    
if __name__ == '__main__':
    if len(sys.argv) > 1:
        env = sys.argv(1)
    else:
        env = 'TEST'
    main(env)

TypeError: 'list' object is not callable

### Using Parallel Processing

In [25]:
from multiprocessing import Process, Manager
import requests

ALL_URLS = [
    'google.com', 'bing.com', 'yahoo.com',
    'twitter.com', 'facebook.com', 'github.com',
    'python.org'
]

def is_up_or_not(url, is_up, lock):
    resp = requests.get(f'http://www.isup.me/{url}')
    if 'is up.' in resp.content:
        is_up.append(url)
    else:
        with lock:
            print(f'HOLY CRAP {url} is down!!!!!')


def get_procs(is_up, lock):
    procs = []
    for url in ALL_URLS:
        procs.append(Process(target=is_up_or_not,
                             args=(url, is_up, lock)))
    return procs


def main():
    manager = Manager()
    is_up = manager.list()
    lock = manager.Lock()
    for p in get_procs(is_up, lock):
        p.start()
        p.join()
    print(is_up)


main()

[]
